In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox
import geopandas as gpd
import osmium
from shapely.geometry import Polygon
import json
import fiona
import os
from shapely.geometry import shape 


for module in [pd, gpd]:
    print(module.__name__, module.__version__)

pandas 1.2.0
geopandas 0.9.0


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#os.mkdir('map_data')

In [44]:
%%bash
wget https://download.bbbike.org/osm/extract/planet_-9.48,38.673_-9.027,38.891.osm.pbf \
    --quiet -O map_data/Porto.osm.pbf

In [46]:
%%bash
wget https://download.bbbike.org/osm/extract/planet_-8.673_41.148_5f89bc30.osm.pbf \
    --quiet -O map_data/Lisbon.osm.pbf

# Loading Data 

## Lisbon

In [47]:
!ogrinfo map_data/Lisbon.osm.pbf

INFO: Open of `map_data/Lisbon.osm.pbf'
      using driver `OSM' successful.
1: points (Point)
2: lines (Line String)
3: multilinestrings (Multi Line String)
4: multipolygons (Multi Polygon)
5: other_relations (Geometry Collection)


In [48]:
%%bash
ogr2ogr -f "GPKG" \
    map_data/lisbon_polygons.gpkg \
    map_data/Lisbon.osm.pbf \
    -nlt POLYGONS \
    -nln polygons

0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: A geometry of type POINT is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type LINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTILINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver w

In [49]:
#Read data
layer_file = "map_data/lisbon_polygons.gpkg"
collection = list(fiona.open(layer_file,'r'))
df1 = pd.DataFrame(collection)


#Check Geometry
def isvalid(geom):
    try:
        shape(geom)
        return 1
    except:
        return 0

df1['isvalid'] = df1['geometry'].apply(lambda x: isvalid(x))
df1 = df1[df1['isvalid'] == 1]
collection = json.loads(df1.to_json(orient='records'))

#Convert to geodataframe
gdf_lis_poly = gpd.GeoDataFrame.from_features(collection)

In [50]:
gdf_lis_poly

,geometry,osm_id,name,amenity,barrier,highway,ref,address,is_in,place,man_made,other_tags,waterway,aerialway,z_order,type,osm_way_id,aeroway,admin_level,boundary,building,craft,geological,historic,land_area,landuse,leisure,military,natural,office,shop,sport,tourism
0,POINT (-8.58577 41.14613),25440127,None,None,None,None,None,None,None,None,None,"""railway""=>""switch""",None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,POINT (-8.64181 41.16761),25504013,None,None,None,give_way,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,POINT (-8.58357 41.16624),25507552,Mercado Abastecedor,None,None,motorway_junction,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,POINT (-8.58266 41.16575),25507555,None,None,None,None,None,None,None,None,None,"""maxspeed""=>""80"",""traffic_sign""=>""maxspeed""",None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,POINT (-8.58106 41.16261),25507559,Merc. Abastecedor,None,None,motorway_junction,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77881,GEOMETRYCOLLECTION EMPTY,12146557,None,None,None,None,None,None,None,None,None,"""restriction""=>""only_straight_on""",None,None,NaN,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
77882,GEOMETRYCOLLECTION EMPTY,12146558,None,None,None,None,None,None,None,None,None,"""restriction""=>""only_straight_on""",None,None,NaN,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
77883,GEOMETRYCOLLECTION EMPTY,12146559,None,None,None,None,None,None,None,None,None,"""restriction""=>""no_u_turn""",None,None,NaN,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
77884,GEOMETRYCOLLECTION EMPTY,12146560,None,None,None,None,None,None,None,None,None,"""restriction""=>""no_u_turn""",None,None,NaN,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [51]:
gdf_lis_poly.columns

Index(['geometry', 'osm_id', 'name', 'amenity', 'barrier', 'highway', 'ref',
       'address', 'is_in', 'place', 'man_made', 'other_tags', 'waterway',
       'aerialway', 'z_order', 'type', 'osm_way_id', 'aeroway', 'admin_level',
       'boundary', 'building', 'craft', 'geological', 'historic', 'land_area',
       'landuse', 'leisure', 'military', 'natural', 'office', 'shop', 'sport',
       'tourism'],
      dtype='object')

## Porto

In [52]:
!ogrinfo map_data/Porto.osm.pbf

INFO: Open of `map_data/Porto.osm.pbf'
      using driver `OSM' successful.
1: points (Point)
2: lines (Line String)
3: multilinestrings (Multi Line String)
4: multipolygons (Multi Polygon)
5: other_relations (Geometry Collection)


In [53]:
%%bash
ogr2ogr -f "GPKG" \
    map_data/porto_polygons.gpkg \
    map_data/Porto.osm.pbf \
    -nlt POLYGONS \
    -nln polygons

0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: A geometry of type POINT is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type LINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTILINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver w

In [54]:
#Read data
layer_file = "map_data/porto_polygons.gpkg"
collection = list(fiona.open(layer_file,'r'))
df1 = pd.DataFrame(collection)


#Check Geometry
def isvalid(geom):
    try:
        shape(geom)
        return 1
    except:
        return 0

df1['isvalid'] = df1['geometry'].apply(lambda x: isvalid(x))
df1 = df1[df1['isvalid'] == 1]
collection = json.loads(df1.to_json(orient='records'))

#Convert to geodataframe
gdf_porto_poly = gpd.GeoDataFrame.from_features(collection)

In [55]:
gdf_porto_poly

,geometry,osm_id,name,amenity,barrier,highway,ref,address,is_in,place,man_made,other_tags,waterway,aerialway,z_order,type,osm_way_id,aeroway,admin_level,boundary,building,craft,geological,historic,land_area,landuse,leisure,military,natural,office,shop,sport,tourism
0,POINT (-9.18832 38.74858),21272086,None,None,None,crossing,None,None,None,None,None,"""crossing""=>""uncontrolled"",""crossing_ref""=>""ze...",None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,POINT (-9.45296 38.75312),21404046,None,None,None,bus_stop,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,POINT (-9.45655 38.75241),21404049,None,None,None,bus_stop,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,POINT (-9.19059 38.72880),21433772,Avenida Tenente Martins,None,None,None,8703,None,None,None,None,"""bus""=>""yes"",""network""=>""Carris"",""operator""=>""...",None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,POINT (-9.19376 38.72666),21433776,Cruz das Oliveiras,None,None,None,8701,None,None,None,None,"""bus""=>""yes"",""network""=>""Carris"",""operator""=>""...",None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491429,GEOMETRYCOLLECTION EMPTY,12666187,None,None,None,None,None,None,None,None,None,"""restriction""=>""no_u_turn""",None,None,NaN,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
491430,GEOMETRYCOLLECTION EMPTY,12666188,None,None,None,None,None,None,None,None,None,"""restriction""=>""no_u_turn""",None,None,NaN,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
491431,GEOMETRYCOLLECTION EMPTY,12666189,None,None,None,None,None,None,None,None,None,"""restriction""=>""only_straight_on""",None,None,NaN,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
491432,GEOMETRYCOLLECTION EMPTY,12666190,None,None,None,None,None,None,None,None,None,"""restriction""=>""only_straight_on""",None,None,NaN,restriction,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## Cleaning Geodataframes

Columns to keep that can affect the demand for green spaces and the surroundings:
Leisure (dog park, garden, park, playground)
Natural
Shop
Tourism

In [56]:
poi_porto = gdf_porto_poly[['geometry', 'osm_id','name','amenity','natural', 'shop', 'leisure', 'tourism']]
poi_porto

,geometry,osm_id,name,amenity,natural,shop,leisure,tourism
0,POINT (-9.18832 38.74858),21272086,None,None,None,None,None,None
1,POINT (-9.45296 38.75312),21404046,None,None,None,None,None,None
2,POINT (-9.45655 38.75241),21404049,None,None,None,None,None,None
3,POINT (-9.19059 38.72880),21433772,Avenida Tenente Martins,None,None,None,None,None
4,POINT (-9.19376 38.72666),21433776,Cruz das Oliveiras,None,None,None,None,None
...,...,...,...,...,...,...,...,...
491429,GEOMETRYCOLLECTION EMPTY,12666187,None,None,None,None,None,None
491430,GEOMETRYCOLLECTION EMPTY,12666188,None,None,None,None,None,None
491431,GEOMETRYCOLLECTION EMPTY,12666189,None,None,None,None,None,None
491432,GEOMETRYCOLLECTION EMPTY,12666190,None,None,None,None,None,None


In [57]:
poi_lisbon = gdf_lis_poly[['geometry', 'osm_id','name','amenity','natural', 'shop', 'leisure', 'tourism']]
poi_lisbon

,geometry,osm_id,name,amenity,natural,shop,leisure,tourism
0,POINT (-8.58577 41.14613),25440127,None,None,None,None,None,None
1,POINT (-8.64181 41.16761),25504013,None,None,None,None,None,None
2,POINT (-8.58357 41.16624),25507552,Mercado Abastecedor,None,None,None,None,None
3,POINT (-8.58266 41.16575),25507555,None,None,None,None,None,None
4,POINT (-8.58106 41.16261),25507559,Merc. Abastecedor,None,None,None,None,None
...,...,...,...,...,...,...,...,...
77881,GEOMETRYCOLLECTION EMPTY,12146557,None,None,None,None,None,None
77882,GEOMETRYCOLLECTION EMPTY,12146558,None,None,None,None,None,None
77883,GEOMETRYCOLLECTION EMPTY,12146559,None,None,None,None,None,None
77884,GEOMETRYCOLLECTION EMPTY,12146560,None,None,None,None,None,None


In [58]:
def remove_rows(df):
    df_mod = df.dropna(how='all', subset=['amenity', 'natural', 'shop', 'leisure', 'tourism'])
    return df_mod

In [62]:
poi_lisbon_clean = remove_rows(poi_lisbon)
poi_lisbon_clean

,geometry,osm_id,name,amenity,natural,shop,leisure,tourism
182,POINT (-8.63455 41.14796),285949956,Repsol,fuel,None,None,None,None
258,POINT (-8.60656 41.16906),406362828,None,fuel,None,None,None,None
267,POINT (-8.60369 41.16168),411614908,None,post_office,None,None,None,None
268,POINT (-8.59840 41.16175),411614923,Estação dos Correios de Eça de Queirós,post_office,None,None,None,None
270,POINT (-8.59847 41.16124),411627955,Santander Totta,bank,None,None,None,None
...,...,...,...,...,...,...,...,...
77600,"POLYGON ((-8.59621 41.12671, -8.59621 41.12665...",None,None,None,None,None,swimming_pool,None
77601,"POLYGON ((-8.59608 41.12727, -8.59615 41.12701...",None,None,None,None,None,pitch,None
77602,"POLYGON ((-8.59032 41.12243, -8.59041 41.12236...",None,Parque Intermarché,parking,None,None,None,None
77603,"POLYGON ((-8.59047 41.12233, -8.59026 41.12222...",None,Parque Intermarché,parking,None,None,None,None


In [61]:
poi_porto_clean = remove_rows(poi_porto)
poi_porto_clean

,geometry,osm_id,name,amenity,natural,shop,leisure,tourism
37,POINT (-9.12955 38.73075),25877927,None,parking,None,None,None,None
166,POINT (-9.19199 38.78378),145997451,Galp,fuel,None,None,None,None
193,POINT (-9.44778 38.69614),196052478,REPSOL,fuel,None,None,None,None
205,POINT (-9.14826 38.74897),205014261,None,parking_entrance,None,None,None,None
229,POINT (-9.19129 38.78191),206254257,None,parking_entrance,None,None,None,None
...,...,...,...,...,...,...,...,...
489471,"POLYGON ((-9.35189 38.70603, -9.35197 38.70597...",None,None,None,None,None,track,None
489472,"POLYGON ((-9.35285 38.70647, -9.35281 38.70644...",None,None,None,None,None,garden,None
489474,"POLYGON ((-9.35465 38.70761, -9.35458 38.70763...",None,None,None,None,None,garden,None
489476,"POLYGON ((-9.35649 38.70423, -9.35638 38.70427...",None,None,None,None,None,garden,None


## Checking the points and polygons that could prove relevant for attracting people to Green Spaces

In [71]:
poi_lisbon_clean['amenity'].unique()
#need to go through the list and focus only on those that are interesting 
#restaurant, cafe, recycling, police, toilets, school, bus_station, kindergarten,  fountain, bench, drinking water, university
# sport club, concert_hall

array(['fuel', 'post_office', 'bank', 'restaurant', 'telephone',
       'post_box', 'pharmacy', 'cafe', 'taxi', 'parking_entrance',
       'recycling', 'cinema', 'fast_food', 'police', 'parking', 'atm',
       'nightclub', 'bar', 'school', 'car_rental', 'clinic', 'toilets',
       'vending_machine', 'library', 'place_of_worship', 'car_wash',
       'bicycle_rental', 'bus_station', 'bicycle_parking', 'kindergarten',
       'parking_exit', 'pub', 'courthouse', 'clock', 'fountain', 'bench',
       'college', 'hospital', 'elevator', 'townhall', 'waste_basket',
       'parking_access', 'drinking_water', 'university',
       'social_facility', 'waste_disposal', 'veterinary',
       'charging_station', 'ice_cream', 'office', 'dentist', 'doctors',
       'music_school', 'shelter', 'printer', 'community_centre',
       'coworking_space', 'trade_union', 'motorcycle_parking',
       'marketplace', 'ferry_terminal', 'bureau_de_change', 'childcare',
       'biergarten', 'studio', 'theatre', 'ticket

In [72]:
poi_lisbon_clean['natural'].unique()

array([None, 'water', 'scrub', 'beach', 'grassland', 'cliff', 'rock',
       'islet', 'spring', 'wood', 'bare_rock', 'heath', 'grass',
       'shingle', 'mud', 'wetland', 'tree_row', 'fell', 'yes'],
      dtype=object)

In [67]:
poi_lisbon_clean['shop'].unique()

array([None, 'hardware', 'mall', 'electronics', 'department_store',
       'supermarket', 'wholesale', 'car', 'sports', 'art_gallery', 'yes',
       'clothes', 'jewelry', 'alcohol', 'lighting', 'books', 'drugstore',
       'butcher', 'watches', 'interior_decoration', 'lottery', 'music',
       'doityourself', 'perfumery', 'hairdresser', 'convenience',
       'bakery', 'swimming_pool', 'shoes', 'frame', 'kiosk',
       'medical_supply', 'optician', 'mobile_phone', 'toys',
       'greengrocer', 'travel_agency', 'furniture', 'hifi', 'leather',
       'fabric', 'houseware', 'baby_goods', 'bicycle', 'florist',
       'pawnbroker', 'gift', 'garden_centre', 'car_repair', 'art',
       'antiques', 'laundry', 'photo', 'Borrachas_e_Plasticos', 'optics',
       'electrical', 'chemist', 'furniture;charity', 'religion',
       'locksmith', 'fireplace', 'wine', 'rental', 'ticket',
       'variety_store', 'copyshop'], dtype=object)

In [73]:
poi_lisbon_clean['leisure'].unique()

array([None, 'sports_centre', 'garden', 'park', 'stadium', 'track',
       'dog_park', 'pitch', 'fitness_centre', 'sports_hall', 'playground',
       'swimming_pool', 'bandstand', 'recreation_ground', 'common',
       'miniature_golf', 'marina', 'yes', 'horse_riding', 'bleachers',
       'outdoor_seating', 'fitness_station'], dtype=object)

In [74]:
poi_lisbon_clean['tourism'].unique()

array([None, 'attraction', 'museum', 'hotel', 'guest_house', 'hostel',
       'artwork', 'yes', 'apartment', 'viewpoint', 'picnic_site'],
      dtype=object)